In [13]:
import numpy as np
import pandas as pd
import seaborn as sns
import os

In [23]:
# Define the directory containing Excel files
directory = "../../data/"

# Get all Excel files in the directory
file_paths = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith(".xlsx")]
column_patterns = ['ΑΙΤΗΣΗ', 'ΜΗΤΡΩΟ', 'ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ.*', 'ΕΤΑΙΡΕΙΑ', 'ΗΜΕΡ.*ΕΚΔ.*ΠΑΡΑΓ.*', 'ΗΜΕΡ.*ΛΗΞΗ.*ΠΑΡΑΓ.*', 'ΠΕΡΙΦΕΡΕΙΑ', 'ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ', 'ΔΗΜΟ.* ', 'ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ', 'ΘΕΣΗ', '.*(MW)', 'ΤΕΧΝΟΛ.*'] 

# Sort file paths based on their names to process them in order
file_paths.sort()



In [24]:
def merge_dataframes(df1, df2, column_patterns):
    """
    Merge the first 13 columns of two DataFrames.

    Parameters:
    df1 (DataFrame): First DataFrame.
    df2 (DataFrame): Second DataFrame.

    Returns:
    DataFrame: Merged DataFrame containing the first 13 columns of each DataFrame.
    """
    # Determine the number of columns to merge (minimum between the two DataFrames)
    num_columns_to_merge = max(min(df1.shape[1], df2.shape[1]), len(column_patterns))
    # Select the first 13 columns from each DataFrame
    df1_subset = df1.iloc[:, :num_columns_to_merge]
    df2_subset = df2.iloc[:, :num_columns_to_merge]

    # Merge the subsets along columns
    merged_df = pd.concat([df1_subset, df2_subset], axis=0)

    return merged_df

def merge_dataframes_with_previous_files(file_paths, column_patterns):
    """
    Merge the first 13 columns of DataFrames from multiple Excel files with two sheets each.
    The column names from the first DataFrame are kept.

    Parameters:
    file_paths (list): List of file paths to Excel files.

    Returns:
    DataFrame: Merged DataFrame containing the first 13 columns of each DataFrame from all files.
    """
    merged_df = None
    merged_df_temp = None
    
    # Iterate over each file path
    for file_path in file_paths:
        print(f"\nLoading file : {file_path}")
        # Load Excel file into separate DataFrames
        df = pd.read_excel(file_path, header=1, sheet_name=[0], parse_dates=True)
        print(f"Number of sheets detected: {len(df.keys())}")

        # Merge in sheet level
        for sheet_name, df_sheet in df.items():
            # Filter according to pattern columns to keep
            df_sheet = df_sheet.filter(regex='|'.join(column_patterns))
            print(f"Number of columns detected in {sheet_name} : {len(df_sheet.columns)}")
            if merged_df_temp is None:
                # Merge the Sheet DataFrames
                merged_df_temp = df_sheet
            else:
                merged_df_temp = merge_dataframes(merged_df_temp, df_sheet, column_patterns)

        # Merge in file level
        # If merged_df is None, assign it the merged DataFrame, else merge with the previous merged DataFrame
        if merged_df is None:
            merged_df = merged_df_temp
        else:
            merged_df = merge_dataframes(merged_df, merged_df_temp, column_patterns)

    return merged_df

In [25]:
# Merge DataFrames from all Excel files
merged_df = merge_dataframes_with_previous_files(file_paths, column_patterns)


Loading file : ../../data/202012.xlsx
Number of sheets detected: 1
Number of columns detected in 0 : 13

Loading file : ../../data/202103.xlsx
Number of sheets detected: 1
Number of columns detected in 0 : 13

Loading file : ../../data/202105.xlsx
Number of sheets detected: 1
Number of columns detected in 0 : 13

Loading file : ../../data/202108.xlsx
Number of sheets detected: 1
Number of columns detected in 0 : 13

Loading file : ../../data/202110.xlsx
Number of sheets detected: 1
Number of columns detected in 0 : 13

Loading file : ../../data/202203.xlsx
Number of sheets detected: 1
Number of columns detected in 0 : 13

Loading file : ../../data/202209.xlsx
Number of sheets detected: 1
Number of columns detected in 0 : 13

Loading file : ../../data/202212.xlsx
Number of sheets detected: 1
Number of columns detected in 0 : 13

Loading file : ../../data/202301.xlsx
Number of sheets detected: 1
Number of columns detected in 0 : 13

Loading file : ../../data/202302.xlsx
Number of sheets

In [27]:
merged_df

,ΑΙΤΗΣΗ,ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ,ΕΤΑΙΡΕΙΑ,ΑΡ. ΜΗΤΡΩΟΥ ΑΔΕΙΩΝ ΡΑΕ,ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ,ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ,ΠΕΡΙΦΕΡΕΙΑ,ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ,ΔΗΜΟΣ,ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ,ΘΕΣΗ,ΙΣΧΥΣ (MW),ΤΕΧΝΟΛΟΓΙΑ
0,B-00279,2001-02-19,ΔΕΗ ΑΝΑΝΕΩΣΙΜΕΣ ΑΕ,ΑΔ-00488,2003-03-04,2028-03-04 00:00:00,ΒΟΡΕΙΟΥ ΑΙΓΑΙΟΥ,ΛΕΣΒΟΥ,ΛΕΣΒΟΥ,ΠΕΤΡΑΣ,ΣΤΥΨΗ/ΠΕΡΙΟΧΗ ΑΡΓΕΝΟΥ,8.000,ΓΕΩΘΕΡΜΙΑ
1,Γ-00798,2004-11-18,ΤΕΡΝΑ ΕΝΕΡΓΕΙΑΚΗ ΑΙ ΓΙΩΡΓΗΣ ΑΕ,ΑΔ-00911,2006-03-23,2021-03-23 00:00:00,ΑΤΤΙΚΗΣ,ΝΗΣΩΝ,ΥΔΡΑΣ,ΥΔΡΑΣ,ΝΗΣΙΔΑ ΑΓΙΟΣ ΓΕΩΡΓΙΟΣ,45.000,ΑΙΟΛΙΚΑ
2,Γ-01329,2006-01-24,ΑΙΟΛΙΚΑ ΠΑΡΚΑ ΠΑΛΙΟΠΥΡΓΟΣ ΑΕ,ΑΔ-00981,2006-10-02,2021-10-02 00:00:00,ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ,ΕΥΒΟΙΑΣ,ΚΑΡΥΣΤΟΥ,ΚΑΡΥΣΤΟΥ & ΚΑΦΗΡΕΩΣ,ΠΑΛΙΟΠΥΡΓΟΣ,13.800,ΑΙΟΛΙΚΑ
3,Γ-01089,2005-07-22,ΤΕΡΝΑ ΕΝΕΡΓΕΙΑΚΗ ΑΙ ΓΙΩΡΓΗΣ ΑΕ,ΑΔ-00989,2006-10-16,2021-10-16 00:00:00,ΑΤΤΙΚΗΣ,ΝΗΣΩΝ,ΥΔΡΑΣ,ΥΔΡΑΣ,ΝΗΣΙΔΑ ΑΓΙΟΣ ΓΕΩΡΓΙΟΣ,24.000,ΑΙΟΛΙΚΑ
4,Γ-01328,2006-01-24,ΑΙΟΛΙΚΑ ΠΑΡΚΑ ΠΛΑΤΑΝΟΣ ΜΟΝΟΠΡΟΣΩΠΗ ΑΕ,ΑΔ-01257,2009-06-16,2034-06-16 00:00:00,ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ,ΕΥΒΟΙΑΣ,ΚΑΡΥΣΤΟΥ,ΜΑΡΜΑΡΙΟΥ,ΠΛΑΤΑΝΟΣ,13.800,ΑΙΟΛΙΚΑ
...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,Γ-011373_Α,2020-12-16,ΑΙΟΛΙΚΗ ΘΕΟΔΩΡΩΝ ΑΝΩΝΥΜΗ ΒΙΟΜΗΧΑΝΙΚΗ ΚΑΙ ΕΝΕΡ...,ΑΔ-04888,2022-11-04,2047-11-04 00:00:00,ΠΕΛΟΠΟΝΝΗΣΟΥ,ΑΡΚΑΔΙΑΣ,"ΜΕΓΑΛΟΠΟΛΗΣ,ΓΟΡΤΥΝΙΑΣ,ΤΡΙΠΟΛΗΣ","ΜΕΓΑΛΟΠΟΛΗΣ,ΤΡΙΚΟΛΩΝΩΝ,ΦΑΛΑΝΘΟΥ","ΜΠΛΕΣΙΒΟΣ, ΨΑΡΟΒΟΥΝΙ, ΛΙΜΝΕΣ ΡΟΔΙΑΣ, ΔΙΑΚΟΠΙ, ...",187.110,ΑΙΟΛΙΚΑ
194,Γ-011380_Α,2020-12-16,ΑΙΟΛΙΚΗ ΘΕΟΔΩΡΩΝ ΑΝΩΝΥΜΗ ΒΙΟΜΗΧΑΝΙΚΗ ΚΑΙ ΕΝΕΡ...,ΑΔ-04889,2022-11-04,2047-11-04 00:00:00,"ΔΥΤΙΚΗΣ ΕΛΛΑΔΑΣ,ΠΕΛΟΠΟΝΝΗΣΟΥ","ΗΛΕΙΑΣ,ΑΡΚΑΔΙΑΣ,ΜΕΣΣΗΝΙΑΣ","ΑΝΔΡΙΤΣΑΙΝΑΣ - ΚΡΕΣΤΕΝΩΝ,ΜΕΓΑΛΟΠΟΛΗΣ,ΟΙΧΑΛΙΑΣ,...","ΑΛΙΦΕΙΡΑΣ,ΑΝΔΡΙΤΣΑΙΝΗΣ,ΓΟΡΤΥΝΟΣ,ΕΙΡΑΣ,ΖΑΧΑΡΩΣ,...","ΓΥΜΝΟΡΡΑΧΗ, ΡΟΣΚΙΟ, ΜΠΡΙΝΙΑ, ΜΠΑΝΤΑΒΑΣ, ΚΟΥΚΟΥ...",417.600,ΑΙΟΛΙΚΑ
195,Γ-011383_Α,2020-12-16,ΑΙΟΛΙΚΗ ΘΕΟΔΩΡΩΝ ΑΝΩΝΥΜΗ ΒΙΟΜΗΧΑΝΙΚΗ ΚΑΙ ΕΝΕΡ...,ΑΔ-04890,2022-11-04,2047-11-04 00:00:00,"ΠΕΛΟΠΟΝΝΗΣΟΥ,ΔΥΤΙΚΗΣ ΕΛΛΑΔΑΣ","ΑΡΚΑΔΙΑΣ,ΑΧΑΪΑΣ,ΚΟΡΙΝΘΙΑΣ","ΓΟΡΤΥΝΙΑΣ,ΚΑΛΑΒΡΥΤΩΝ,ΤΡΙΠΟΛΗΣ,ΕΡΥΜΑΝΘΟΥ,ΣΙΚΥΩΝΙΩΝ","ΒΥΤΙΝΑΣ,ΚΑΛΑΒΡΥΤΩΝ,ΚΛΕΙΤΟΡΙΑΣ, ΚΛΕΙΤΟΡΟΣ,ΚΟΝ...","ΛΑΚΩΜΑΤΑ, ΠΑΝΑΓΙΑ, ΣΑΙΤΑΣ, ΔΡΑΚΟΒΟΥΝΙ, ΒΥΘΟΥΛΑ...",280.665,ΑΙΟΛΙΚΑ
196,Γ-013272,2021-06-09,VOLTON ΕΛΛΗΝΙΚΗ ΕΝΕΡΓΕΙΑΚΗ ΑΝΩΝΥΜΗ ΕΤΑΙΡΕΙΑ,ΑΔ-04891,2022-11-04,2047-11-04 00:00:00,ΠΕΛΟΠΟΝΝΗΣΟΥ,"ΑΡΓΟΛΙΔΑΣ, ΑΡΚΑΔΙΑΣ","ΑΡΓΟΥΣ -ΜΥΚΗΝΩΝ, ΤΡΙΠΟΛΗΣ, ΒΟΡΕΙΑΣ ΚΥΝΟΥΡΙΑΣ, ...","ΑΡΓΟΥΣ, ΒΑΛΤΕΤΣΙΟΥ, ΒΟΡΕΙΑΣ ΚΥΝΟΥΡΙΑΣ, ΛΥΡΚΕΙΑ...",ΑΓΙΟΣ ΠΕΤΡΟΣ-ΒΑΛΤΕΤΣΙ-ΝΕΣΤΑΝΗ-ΤΣΕΜΠΕΡΟΥ,298.200,ΑΙΟΛΙΚΑ


In [28]:
def convert_to_datetime(df):
    """
    Convert date-like columns in a DataFrame to datetime type.

    Parameters:
    df (DataFrame): Input DataFrame.

    Returns:
    DataFrame: DataFrame with date-like columns converted to datetime type.
    """
    # Copy the DataFrame to avoid modifying the original DataFrame
    df_copy = df.copy()
    
    # Iterate through each column
    for col in df_copy.columns:
        print(f"Attempting column: {col}")
        # Check if the column can be converted to datetime
        try:
            df_copy[col] = pd.to_datetime(df_copy[col], format="%Y-%m-%d")
            print(f"Column {col} transformed succesfully.\n")
        except:
            print(f"Column {col} is incompatible with datetime.\n")
            # If conversion fails, skip to the next column
            continue
    return df_copy

In [29]:
def assign_naming(df, colnames=None):

    if colnames is None:
        colnames = ['ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ', 'ΕΤΑΙΡΕΙΑ', 'ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ',
        'ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ', 'ΠΕΡΙΦΕΡΕΙΑ', 'ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ',
        'ΔΗΜΟ.* ', 'ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ', 'ΘΕΣΗ', '(MW)', 'ΤΕΧΝΟΛΟΓΙΑ']

    num_columns_to_merge = max(len(df.columns), 13)

    df = df.set_axis(colnames[:num_columns_to_merge], axis='columns')

    return df

In [30]:
# Define pairs of columns to be merged
# merge_pairs = {
#     'ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ': 'ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ',
#     'ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ': 'ΗΜΕΡ. ΕΚΔΟΣΗΣ ΒΕΒΑΙΩΣΗΣ ΠΑΡΑΓΩΓΟΥ',
#     # 'ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ': 'ΗΜΕΡ. ΛΗΞΗΣ ΒΕΒΑΙΩΣΗΣ ΠΑΡΑΓΩΓΟΥ',
#     # 'ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ': 'ΗΜΕΡ. ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ',
#     # 'ΙΣΧΥΣ (MW)': 'Ισχύς (MW)',
#     # 'ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ' : 'ΗΜΕΡ. ΕΚΔΟΣΗΣ ΒΕΒΑΙΩΣΗΣ ΠΑΡΑΓΩΓΟΥ',
#     # 'ΤΕΧΝΟΛΟΓΙΑ' : 'ΤΕΧΝΟΛΟΓΙΑ ΑΠΕ',
#     # 'ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ' : 'ΗΜΕΡ. ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ',
#     # 'ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ' : 'ΗΜΕΡ. ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ'
# }

def merge_columns(df, merge_pairs):
    for col_to_assign, col_to_del in merge_pairs.items():
        # Filter rows where values exist in the column to assign
        rows_with_values = df[col_to_del].notna()
        # Fill NaN values in the column to assign with values from the column to delete only where values exist
        df.loc[rows_with_values, col_to_assign] = df.loc[rows_with_values, col_to_del]
        # Drop the column to delete
        df.drop(columns=col_to_del, inplace=True)
    return df


df_merged_test = convert_to_datetime(merged_df.copy())
# Merge columns based on merge pairs
# df_merged_test = merge_columns(df_merged_test.copy(), merge_pairs)

Attempting column: ΑΙΤΗΣΗ
Column ΑΙΤΗΣΗ is incompatible with datetime.

Attempting column: ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ
Column ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ transformed succesfully.

Attempting column: ΕΤΑΙΡΕΙΑ
Column ΕΤΑΙΡΕΙΑ is incompatible with datetime.

Attempting column: ΑΡ. ΜΗΤΡΩΟΥ ΑΔΕΙΩΝ ΡΑΕ
Column ΑΡ. ΜΗΤΡΩΟΥ ΑΔΕΙΩΝ ΡΑΕ is incompatible with datetime.

Attempting column: ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ
Column ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ transformed succesfully.

Attempting column: ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ
Column ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ is incompatible with datetime.

Attempting column: ΠΕΡΙΦΕΡΕΙΑ
Column ΠΕΡΙΦΕΡΕΙΑ is incompatible with datetime.

Attempting column: ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ
Column ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ is incompatible with datetime.

Attempting column: ΔΗΜΟΣ 
Column ΔΗΜΟΣ  is incompatible with datetime.

Attempting column: ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ
Column ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ is incompatible with datetime.

Attempting column: ΘΕΣΗ
Column ΘΕΣΗ is incompatible with datetime.



In [38]:
df_merged_test[df_merged_test['ΑΙΤΗΣΗ'] == 'B-00279']

,ΑΙΤΗΣΗ,ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ,ΕΤΑΙΡΕΙΑ,ΑΡ. ΜΗΤΡΩΟΥ ΑΔΕΙΩΝ ΡΑΕ,ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ,ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ,ΠΕΡΙΦΕΡΕΙΑ,ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ,ΔΗΜΟΣ,ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ,ΘΕΣΗ,ΙΣΧΥΣ (MW),ΤΕΧΝΟΛΟΓΙΑ
0,B-00279,2001-02-19,ΔΕΗ ΑΝΑΝΕΩΣΙΜΕΣ ΑΕ,ΑΔ-00488,2003-03-04,2028-03-04 00:00:00,ΒΟΡΕΙΟΥ ΑΙΓΑΙΟΥ,ΛΕΣΒΟΥ,ΛΕΣΒΟΥ,ΠΕΤΡΑΣ,ΣΤΥΨΗ/ΠΕΡΙΟΧΗ ΑΡΓΕΝΟΥ,8.0,ΓΕΩΘΕΡΜΙΑ
0,B-00279,2001-02-19,ΔΕΗ ΑΝΑΝΕΩΣΙΜΕΣ ΑΕ,ΑΔ-00488,2003-03-04,2028-03-04 00:00:00,ΒΟΡΕΙΟΥ ΑΙΓΑΙΟΥ,ΛΕΣΒΟΥ,ΛΕΣΒΟΥ,ΠΕΤΡΑΣ,ΣΤΥΨΗ/ΠΕΡΙΟΧΗ ΑΡΓΕΝΟΥ,8.0,ΓΕΩΘΕΡΜΙΑ
0,B-00279,2001-02-19,ΔΕΗ ΑΝΑΝΕΩΣΙΜΕΣ ΑΕ,ΑΔ-00488,2003-03-04,2028-03-04 00:00:00,ΒΟΡΕΙΟΥ ΑΙΓΑΙΟΥ,ΛΕΣΒΟΥ,ΛΕΣΒΟΥ,ΠΕΤΡΑΣ,ΣΤΥΨΗ/ΠΕΡΙΟΧΗ ΑΡΓΕΝΟΥ,8.0,ΓΕΩΘΕΡΜΙΑ
0,B-00279,2001-02-19,ΔΕΗ ΑΝΑΝΕΩΣΙΜΕΣ ΑΕ,ΑΔ-00488,2003-03-04,2028-03-04 00:00:00,ΒΟΡΕΙΟΥ ΑΙΓΑΙΟΥ,ΛΕΣΒΟΥ,ΛΕΣΒΟΥ,ΠΕΤΡΑΣ,ΣΤΥΨΗ/ΠΕΡΙΟΧΗ ΑΡΓΕΝΟΥ,8.0,ΓΕΩΘΕΡΜΙΑ
0,B-00279,2001-02-19,ΔΕΗ ΑΝΑΝΕΩΣΙΜΕΣ ΑΕ,ΑΔ-00488,2003-03-04,2028-03-04 00:00:00,ΒΟΡΕΙΟΥ ΑΙΓΑΙΟΥ,ΛΕΣΒΟΥ,ΛΕΣΒΟΥ,ΠΕΤΡΑΣ,ΣΤΥΨΗ/ΠΕΡΙΟΧΗ ΑΡΓΕΝΟΥ,8.0,ΓΕΩΘΕΡΜΙΑ
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,B-00279,2001-02-19,ΔΕΗ ΑΝΑΝΕΩΣΙΜΕΣ ΑΕ,ΑΔ-00488,2003-03-04,2028-03-04 00:00:00,ΒΟΡΕΙΟΥ ΑΙΓΑΙΟΥ,ΛΕΣΒΟΥ,ΛΕΣΒΟΥ,ΠΕΤΡΑΣ,ΣΤΥΨΗ/ΠΕΡΙΟΧΗ ΑΡΓΕΝΟΥ,8.0,ΓΕΩΘΕΡΜΙΑ
0,B-00279,2001-02-19,ΔΕΗ ΑΝΑΝΕΩΣΙΜΕΣ ΑΕ,ΑΔ-00488,2003-03-04,2028-03-04 00:00:00,ΒΟΡΕΙΟΥ ΑΙΓΑΙΟΥ,ΛΕΣΒΟΥ,ΛΕΣΒΟΥ,ΠΕΤΡΑΣ,ΣΤΥΨΗ/ΠΕΡΙΟΧΗ ΑΡΓΕΝΟΥ,8.0,ΓΕΩΘΕΡΜΙΑ
0,B-00279,2001-02-19,ΔΕΗ ΑΝΑΝΕΩΣΙΜΕΣ ΑΕ,ΑΔ-00488,2003-03-04,2028-03-04 00:00:00,ΒΟΡΕΙΟΥ ΑΙΓΑΙΟΥ,ΛΕΣΒΟΥ,ΛΕΣΒΟΥ,ΠΕΤΡΑΣ,ΣΤΥΨΗ/ΠΕΡΙΟΧΗ ΑΡΓΕΝΟΥ,8.0,ΓΕΩΘΕΡΜΙΑ
0,B-00279,2001-02-19,ΓΕΩΘΕΡΜΙΚΟΣ ΣΤΟΧΟΣ ΔΥΟ ΜΟΝΟΠΡΟΣΩΠΗ Α.Ε.,ΑΔ-00488,2003-03-04,2028-03-04 00:00:00,ΒΟΡΕΙΟΥ ΑΙΓΑΙΟΥ,ΛΕΣΒΟΥ,ΛΕΣΒΟΥ,ΠΕΤΡΑΣ,ΣΤΥΨΗ/ΠΕΡΙΟΧΗ ΑΡΓΕΝΟΥ,8.0,ΓΕΩΘΕΡΜΙΑ


In [36]:
df_merged_test.drop_duplicates()

,ΑΙΤΗΣΗ,ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ,ΕΤΑΙΡΕΙΑ,ΑΡ. ΜΗΤΡΩΟΥ ΑΔΕΙΩΝ ΡΑΕ,ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ,ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ,ΠΕΡΙΦΕΡΕΙΑ,ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ,ΔΗΜΟΣ,ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ,ΘΕΣΗ,ΙΣΧΥΣ (MW),ΤΕΧΝΟΛΟΓΙΑ
0,B-00279,2001-02-19 00:00:00.000,ΔΕΗ ΑΝΑΝΕΩΣΙΜΕΣ ΑΕ,ΑΔ-00488,2003-03-04 00:00:00,2028-03-04 00:00:00,ΒΟΡΕΙΟΥ ΑΙΓΑΙΟΥ,ΛΕΣΒΟΥ,ΛΕΣΒΟΥ,ΠΕΤΡΑΣ,ΣΤΥΨΗ/ΠΕΡΙΟΧΗ ΑΡΓΕΝΟΥ,8.0,ΓΕΩΘΕΡΜΙΑ
1,Γ-00798,2004-11-18 00:00:00.000,ΤΕΡΝΑ ΕΝΕΡΓΕΙΑΚΗ ΑΙ ΓΙΩΡΓΗΣ ΑΕ,ΑΔ-00911,2006-03-23 00:00:00,2021-03-23 00:00:00,ΑΤΤΙΚΗΣ,ΝΗΣΩΝ,ΥΔΡΑΣ,ΥΔΡΑΣ,ΝΗΣΙΔΑ ΑΓΙΟΣ ΓΕΩΡΓΙΟΣ,45.0,ΑΙΟΛΙΚΑ
2,Γ-01329,2006-01-24 00:00:00.000,ΑΙΟΛΙΚΑ ΠΑΡΚΑ ΠΑΛΙΟΠΥΡΓΟΣ ΑΕ,ΑΔ-00981,2006-10-02 00:00:00,2021-10-02 00:00:00,ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ,ΕΥΒΟΙΑΣ,ΚΑΡΥΣΤΟΥ,ΚΑΡΥΣΤΟΥ & ΚΑΦΗΡΕΩΣ,ΠΑΛΙΟΠΥΡΓΟΣ,13.8,ΑΙΟΛΙΚΑ
3,Γ-01089,2005-07-22 00:00:00.000,ΤΕΡΝΑ ΕΝΕΡΓΕΙΑΚΗ ΑΙ ΓΙΩΡΓΗΣ ΑΕ,ΑΔ-00989,2006-10-16 00:00:00,2021-10-16 00:00:00,ΑΤΤΙΚΗΣ,ΝΗΣΩΝ,ΥΔΡΑΣ,ΥΔΡΑΣ,ΝΗΣΙΔΑ ΑΓΙΟΣ ΓΕΩΡΓΙΟΣ,24.0,ΑΙΟΛΙΚΑ
4,Γ-01328,2006-01-24 00:00:00.000,ΑΙΟΛΙΚΑ ΠΑΡΚΑ ΠΛΑΤΑΝΟΣ ΜΟΝΟΠΡΟΣΩΠΗ ΑΕ,ΑΔ-01257,2009-06-16 00:00:00,2034-06-16 00:00:00,ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ,ΕΥΒΟΙΑΣ,ΚΑΡΥΣΤΟΥ,ΜΑΡΜΑΡΙΟΥ,ΠΛΑΤΑΝΟΣ,13.8,ΑΙΟΛΙΚΑ
...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,Γ-05948,2015-06-05 00:00:00.000,ΤΕΡΝΑ ΕΝΕΡΓΕΙΑΚΗ ΑΝΩΝΥΜΗ ΒΙΟΜΗΧΑΝΙΚΗ ΕΜΠΟΡΙΚΗ ...,ΑΔ-04526,2021-07-29 00:00:00,2046-07-29 18:00:00,ΣΤ ΕΛΛΑΔΑΣ,ΕΥΒΟΙΑΣ,ΚΥΜΗΣ - ΑΛΙΒΕΡΙΟΥ,ΔΥΣΤΥΩΝ,ΑΓΙΑ ΤΡΙΑΣ,4.4,ΑΙΟΛΙΚΑ
175,Γ-02883,2007-10-09 00:00:00.000,ΕΥΒΟΪΚΟΣ ΑΝΕΜΟΣ ΑΝΩΝΥΜΗ ΕΤΑΙΡΕΙΑ ΠΑΡΑΓΩΓΗΣ ΗΛΕ...,ΑΔ-04529,2021-07-15 00:00:00,2046-07-15 18:00:00,ΣΤ ΕΛΛΑΔΑΣ,ΕΥΒΟΙΑΣ,ΔΙΡΦΥΩΝ - ΜΕΣΣΑΠΙΩΝ,ΔΙΡΦΥΩΝ,ΨΗΛΟΣ ΒΡΑΧΟΣ ΜΕΣΣΑΠΙΩΝ,25.3,ΑΙΟΛΙΚΑ
199,Γ-015033,2023-02-10 17:09:44.508,ENVITEC ΑΝΩΝΥΜΗ ΕΤΑΙΡΕΙΑ ΤΕΧΝΙΚΩΝ ΚΑΙ ΠΕΡΙΒΑΛ...,ΑΔ-010966,2023-12-01 11:41:37,2048-12-01 11:41:37,ΣΤΕΡΕΑΣ ΕΛΛΑΔΑΣ,ΕΥΒΟΙΑΣ,ΚΑΡΥΣΤΟΥ,ΣΤΥΡΕΩΝ,ΤΡΑΒΑ,20.7,ΑΙΟΛΙΚΑ
6,Γ-015033,2023-02-10 17:09:44.508,ENVITEC ΑΝΩΝΥΜΗ ΕΤΑΙΡΕΙΑ ΤΕΧΝΙΚΩΝ ΚΑΙ ΠΕΡΙΒΑΛ...,ΑΔ-010966,2023-12-01 00:00:00,1948-12-01 00:00:00,ΣΤΕΡΕΑΣ ΕΛΛΑΔΑΣ,ΕΥΒΟΙΑΣ,ΚΑΡΥΣΤΟΥ,ΣΤΥΡΕΩΝ,ΤΡΑΒΑ,20.7,ΑΙΟΛΙΚΑ


In [35]:
df_merged_test[df_merged_test.duplicated()]

,ΑΙΤΗΣΗ,ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ,ΕΤΑΙΡΕΙΑ,ΑΡ. ΜΗΤΡΩΟΥ ΑΔΕΙΩΝ ΡΑΕ,ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ,ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ,ΠΕΡΙΦΕΡΕΙΑ,ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ,ΔΗΜΟΣ,ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ,ΘΕΣΗ,ΙΣΧΥΣ (MW),ΤΕΧΝΟΛΟΓΙΑ
0,B-00279,2001-02-19,ΔΕΗ ΑΝΑΝΕΩΣΙΜΕΣ ΑΕ,ΑΔ-00488,2003-03-04,2028-03-04 00:00:00,ΒΟΡΕΙΟΥ ΑΙΓΑΙΟΥ,ΛΕΣΒΟΥ,ΛΕΣΒΟΥ,ΠΕΤΡΑΣ,ΣΤΥΨΗ/ΠΕΡΙΟΧΗ ΑΡΓΕΝΟΥ,8.000,ΓΕΩΘΕΡΜΙΑ
1,Γ-00798,2004-11-18,ΤΕΡΝΑ ΕΝΕΡΓΕΙΑΚΗ ΑΙ ΓΙΩΡΓΗΣ ΑΕ,ΑΔ-00911,2006-03-23,2021-03-23 00:00:00,ΑΤΤΙΚΗΣ,ΝΗΣΩΝ,ΥΔΡΑΣ,ΥΔΡΑΣ,ΝΗΣΙΔΑ ΑΓΙΟΣ ΓΕΩΡΓΙΟΣ,45.000,ΑΙΟΛΙΚΑ
2,Γ-01329,2006-01-24,ΑΙΟΛΙΚΑ ΠΑΡΚΑ ΠΑΛΙΟΠΥΡΓΟΣ ΑΕ,ΑΔ-00981,2006-10-02,2021-10-02 00:00:00,ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ,ΕΥΒΟΙΑΣ,ΚΑΡΥΣΤΟΥ,ΚΑΡΥΣΤΟΥ & ΚΑΦΗΡΕΩΣ,ΠΑΛΙΟΠΥΡΓΟΣ,13.800,ΑΙΟΛΙΚΑ
3,Γ-01089,2005-07-22,ΤΕΡΝΑ ΕΝΕΡΓΕΙΑΚΗ ΑΙ ΓΙΩΡΓΗΣ ΑΕ,ΑΔ-00989,2006-10-16,2021-10-16 00:00:00,ΑΤΤΙΚΗΣ,ΝΗΣΩΝ,ΥΔΡΑΣ,ΥΔΡΑΣ,ΝΗΣΙΔΑ ΑΓΙΟΣ ΓΕΩΡΓΙΟΣ,24.000,ΑΙΟΛΙΚΑ
4,Γ-01328,2006-01-24,ΑΙΟΛΙΚΑ ΠΑΡΚΑ ΠΛΑΤΑΝΟΣ ΜΟΝΟΠΡΟΣΩΠΗ ΑΕ,ΑΔ-01257,2009-06-16,2034-06-16 00:00:00,ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ,ΕΥΒΟΙΑΣ,ΚΑΡΥΣΤΟΥ,ΜΑΡΜΑΡΙΟΥ,ΠΛΑΤΑΝΟΣ,13.800,ΑΙΟΛΙΚΑ
...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,Γ-011373_Α,2020-12-16,ΑΙΟΛΙΚΗ ΘΕΟΔΩΡΩΝ ΑΝΩΝΥΜΗ ΒΙΟΜΗΧΑΝΙΚΗ ΚΑΙ ΕΝΕΡ...,ΑΔ-04888,2022-11-04,2047-11-04 00:00:00,ΠΕΛΟΠΟΝΝΗΣΟΥ,ΑΡΚΑΔΙΑΣ,"ΜΕΓΑΛΟΠΟΛΗΣ,ΓΟΡΤΥΝΙΑΣ,ΤΡΙΠΟΛΗΣ","ΜΕΓΑΛΟΠΟΛΗΣ,ΤΡΙΚΟΛΩΝΩΝ,ΦΑΛΑΝΘΟΥ","ΜΠΛΕΣΙΒΟΣ, ΨΑΡΟΒΟΥΝΙ, ΛΙΜΝΕΣ ΡΟΔΙΑΣ, ΔΙΑΚΟΠΙ, ...",187.110,ΑΙΟΛΙΚΑ
194,Γ-011380_Α,2020-12-16,ΑΙΟΛΙΚΗ ΘΕΟΔΩΡΩΝ ΑΝΩΝΥΜΗ ΒΙΟΜΗΧΑΝΙΚΗ ΚΑΙ ΕΝΕΡ...,ΑΔ-04889,2022-11-04,2047-11-04 00:00:00,"ΔΥΤΙΚΗΣ ΕΛΛΑΔΑΣ,ΠΕΛΟΠΟΝΝΗΣΟΥ","ΗΛΕΙΑΣ,ΑΡΚΑΔΙΑΣ,ΜΕΣΣΗΝΙΑΣ","ΑΝΔΡΙΤΣΑΙΝΑΣ - ΚΡΕΣΤΕΝΩΝ,ΜΕΓΑΛΟΠΟΛΗΣ,ΟΙΧΑΛΙΑΣ,...","ΑΛΙΦΕΙΡΑΣ,ΑΝΔΡΙΤΣΑΙΝΗΣ,ΓΟΡΤΥΝΟΣ,ΕΙΡΑΣ,ΖΑΧΑΡΩΣ,...","ΓΥΜΝΟΡΡΑΧΗ, ΡΟΣΚΙΟ, ΜΠΡΙΝΙΑ, ΜΠΑΝΤΑΒΑΣ, ΚΟΥΚΟΥ...",417.600,ΑΙΟΛΙΚΑ
195,Γ-011383_Α,2020-12-16,ΑΙΟΛΙΚΗ ΘΕΟΔΩΡΩΝ ΑΝΩΝΥΜΗ ΒΙΟΜΗΧΑΝΙΚΗ ΚΑΙ ΕΝΕΡ...,ΑΔ-04890,2022-11-04,2047-11-04 00:00:00,"ΠΕΛΟΠΟΝΝΗΣΟΥ,ΔΥΤΙΚΗΣ ΕΛΛΑΔΑΣ","ΑΡΚΑΔΙΑΣ,ΑΧΑΪΑΣ,ΚΟΡΙΝΘΙΑΣ","ΓΟΡΤΥΝΙΑΣ,ΚΑΛΑΒΡΥΤΩΝ,ΤΡΙΠΟΛΗΣ,ΕΡΥΜΑΝΘΟΥ,ΣΙΚΥΩΝΙΩΝ","ΒΥΤΙΝΑΣ,ΚΑΛΑΒΡΥΤΩΝ,ΚΛΕΙΤΟΡΙΑΣ, ΚΛΕΙΤΟΡΟΣ,ΚΟΝ...","ΛΑΚΩΜΑΤΑ, ΠΑΝΑΓΙΑ, ΣΑΙΤΑΣ, ΔΡΑΚΟΒΟΥΝΙ, ΒΥΘΟΥΛΑ...",280.665,ΑΙΟΛΙΚΑ
196,Γ-013272,2021-06-09,VOLTON ΕΛΛΗΝΙΚΗ ΕΝΕΡΓΕΙΑΚΗ ΑΝΩΝΥΜΗ ΕΤΑΙΡΕΙΑ,ΑΔ-04891,2022-11-04,2047-11-04 00:00:00,ΠΕΛΟΠΟΝΝΗΣΟΥ,"ΑΡΓΟΛΙΔΑΣ, ΑΡΚΑΔΙΑΣ","ΑΡΓΟΥΣ -ΜΥΚΗΝΩΝ, ΤΡΙΠΟΛΗΣ, ΒΟΡΕΙΑΣ ΚΥΝΟΥΡΙΑΣ, ...","ΑΡΓΟΥΣ, ΒΑΛΤΕΤΣΙΟΥ, ΒΟΡΕΙΑΣ ΚΥΝΟΥΡΙΑΣ, ΛΥΡΚΕΙΑ...",ΑΓΙΟΣ ΠΕΤΡΟΣ-ΒΑΛΤΕΤΣΙ-ΝΕΣΤΑΝΗ-ΤΣΕΜΠΕΡΟΥ,298.200,ΑΙΟΛΙΚΑ


Deciding which workspace information to collect

Collecting workspace information

In [33]:
df_merged_test.isna().sum()

ΑΙΤΗΣΗ                             0
ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ      283
ΕΤΑΙΡΕΙΑ                         283
ΑΡ. ΜΗΤΡΩΟΥ ΑΔΕΙΩΝ ΡΑΕ           283
ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ     283
ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ    283
ΠΕΡΙΦΕΡΕΙΑ                       283
ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ             283
ΔΗΜΟΣ                            283
ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ                 283
ΘΕΣΗ                             283
ΙΣΧΥΣ (MW)                       283
ΤΕΧΝΟΛΟΓΙΑ                       283
dtype: int64

In [34]:
merged_df.isna().sum()

ΑΙΤΗΣΗ                             0
ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ      283
ΕΤΑΙΡΕΙΑ                         283
ΑΡ. ΜΗΤΡΩΟΥ ΑΔΕΙΩΝ ΡΑΕ           283
ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ     283
ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ    283
ΠΕΡΙΦΕΡΕΙΑ                       283
ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ             283
ΔΗΜΟΣ                            283
ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ                 283
ΘΕΣΗ                             283
ΙΣΧΥΣ (MW)                       283
ΤΕΧΝΟΛΟΓΙΑ                       283
dtype: int64

In [ ]:
# sample_df = merge_dataframes(sample_df[0], sample_df[1])

In [ ]:
sample_df_dt = convert_to_datetime(sample_df)

Attempting column: ΑΙΤΗΣΗ
Column ΑΙΤΗΣΗ is incompatible with datetime.

Attempting column: ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ
Column ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ transformed succesfully.

Attempting column: ΕΤΑΙΡΕΙΑ
Column ΕΤΑΙΡΕΙΑ is incompatible with datetime.

Attempting column: ΑΡ. ΜΗΤΡΩΟΥ ΑΔΕΙΩΝ ΡΑΕ
Column ΑΡ. ΜΗΤΡΩΟΥ ΑΔΕΙΩΝ ΡΑΕ is incompatible with datetime.

Attempting column: ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ
Column ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ transformed succesfully.

Attempting column: ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ
Column ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ transformed succesfully.

Attempting column: ΠΕΡΙΦΕΡΕΙΑ
Column ΠΕΡΙΦΕΡΕΙΑ is incompatible with datetime.

Attempting column: ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ
Column ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ is incompatible with datetime.

Attempting column: ΔΗΜΟΣ 
Column ΔΗΜΟΣ  is incompatible with datetime.

Attempting column: ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ
Column ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ is incompatible with datetime.

Attempting column: ΘΕΣΗ
Column ΘΕΣΗ is incompatible with datetime.

Attemp

In [ ]:
sample_df_dt.head()

,ΑΙΤΗΣΗ,ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ,ΕΤΑΙΡΕΙΑ,ΑΡ. ΜΗΤΡΩΟΥ ΑΔΕΙΩΝ ΡΑΕ,ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ,ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ,ΠΕΡΙΦΕΡΕΙΑ,ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ,ΔΗΜΟΣ,ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ,ΘΕΣΗ,ΙΣΧΥΣ (MW),ΤΕΧΝΟΛΟΓΙΑ
0,B-00279,2001-02-19,ΔΕΗ ΑΝΑΝΕΩΣΙΜΕΣ ΑΕ,ΑΔ-00488,2003-03-04,2028-03-04,ΒΟΡΕΙΟΥ ΑΙΓΑΙΟΥ,ΛΕΣΒΟΥ,ΛΕΣΒΟΥ,ΠΕΤΡΑΣ,ΣΤΥΨΗ/ΠΕΡΙΟΧΗ ΑΡΓΕΝΟΥ,8.0,ΓΕΩΘΕΡΜΙΑ
1,Γ-00798,2004-11-18,ΤΕΡΝΑ ΕΝΕΡΓΕΙΑΚΗ ΑΙ ΓΙΩΡΓΗΣ ΑΕ,ΑΔ-00911,2006-03-23,2021-03-23,ΑΤΤΙΚΗΣ,ΝΗΣΩΝ,ΥΔΡΑΣ,ΥΔΡΑΣ,ΝΗΣΙΔΑ ΑΓΙΟΣ ΓΕΩΡΓΙΟΣ,45.0,ΑΙΟΛΙΚΑ
2,Γ-01329,2006-01-24,ΑΙΟΛΙΚΑ ΠΑΡΚΑ ΠΑΛΙΟΠΥΡΓΟΣ ΑΕ,ΑΔ-00981,2006-10-02,2021-10-02,ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ,ΕΥΒΟΙΑΣ,ΚΑΡΥΣΤΟΥ,ΚΑΡΥΣΤΟΥ & ΚΑΦΗΡΕΩΣ,ΠΑΛΙΟΠΥΡΓΟΣ,13.8,ΑΙΟΛΙΚΑ
3,Γ-01089,2005-07-22,ΤΕΡΝΑ ΕΝΕΡΓΕΙΑΚΗ ΑΙ ΓΙΩΡΓΗΣ ΑΕ,ΑΔ-00989,2006-10-16,2021-10-16,ΑΤΤΙΚΗΣ,ΝΗΣΩΝ,ΥΔΡΑΣ,ΥΔΡΑΣ,ΝΗΣΙΔΑ ΑΓΙΟΣ ΓΕΩΡΓΙΟΣ,24.0,ΑΙΟΛΙΚΑ
4,Γ-01328,2006-01-24,ΑΙΟΛΙΚΑ ΠΑΡΚΑ ΠΛΑΤΑΝΟΣ ΜΟΝΟΠΡΟΣΩΠΗ ΑΕ,ΑΔ-01257,2009-06-16,2034-06-16,ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ,ΕΥΒΟΙΑΣ,ΚΑΡΥΣΤΟΥ,ΜΑΡΜΑΡΙΟΥ,ΠΛΑΤΑΝΟΣ,13.8,ΑΙΟΛΙΚΑ


### Automate the whole thing for various files